## Generator 可以使用 next 來進行循環中的一步
文字上有點難解釋，直接來看範例就能了解什麼是 Generator!

### 撰寫一個 Generator，一次吐出 list 中的一個值

In [1]:
def output_from_list_generator(your_list):
    for i in your_list:
        yield i 

In [2]:
my_list = [1, 2, 3, 4, 5]

In [3]:
gen = output_from_list_generator(my_list)

In [4]:
gen

<generator object output_from_list_generator at 0x000001AE8369E360>

In [5]:
print(next(gen))

1


In [6]:
print(next(gen))

2


In [7]:
print(next(gen))

3


In [8]:
print(next(gen))

4


In [9]:
print(next(gen))

5


In [10]:
print(next(gen))

StopIteration: 

### 從上面的範例程式碼我們可以看到，當使用一次 next，generator 就會跑 for_loop 一次，因此得到 list 中的第一個值，當再使用一次後，for_loop 記得上次的循環，所以吐出第二個值。最後一次，因為 for loop 已經執行結束了，所以再使用 next 就會看到 StopIteration，無法在得到值

### 我們可以撰寫一個無限循環的 Generator，只要使用 While True 即可

In [11]:
def inf_loop_generator(your_list):
    while True:
        for i in your_list:
            yield i

In [12]:
gen = inf_loop_generator(my_list)

In [13]:
print(next(gen))

1


In [14]:
print(next(gen))

2


In [15]:
print(next(gen))

3


In [16]:
print(next(gen))

4


In [17]:
print(next(gen))

5


In [18]:
print(next(gen))

1


In [19]:
print(next(gen))

2


### 上面的程式碼因為我們使用了 While True，所以 for loop 不會結束，只要 call next 就一定會跑一次循環，並返回值

## 雖然 Cifar-10 的資料可以全部讀進記憶體，但讓我們試著用 Generator，批次的把 Cifar 10 的資料取出來，一次取 32 張出來！

In [20]:
def img_combine(img, ncols=8, size=1, path=False):
    from math import ceil
    import matplotlib.pyplot as plt
    import numpy as np
    nimg = len(img)
    nrows = int(ceil(nimg/ncols))
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharex=True, sharey=True, figsize=(ncols*size,nrows*size))
    if nrows == 0:
        return
    elif ncols == 1:
        for r, ax in zip(np.arange(nrows), axes):
            nth=r
            if nth < nimg:
                ax.imshow(img[nth], cmap='rainbow', vmin=0, vmax=1)
                
            ax.set_axis_off()
    elif nrows == 1:
        for c, ax in zip(np.arange(ncols), axes):
            nth=c
            if nth < nimg:
                ax.imshow(img[nth], cmap='rainbow', vmin=0, vmax=1)
            ax.set_axis_off()
    else:
        for r, row in zip(np.arange(nrows), axes):
            for c, ax in zip(np.arange(ncols), row):
                nth=r*ncols+c
                if nth < nimg:
                    ax.imshow(img[nth], cmap='rainbow', vmin=0, vmax=1)
                ax.set_axis_off()
    plt.show()

In [21]:
from keras.datasets import cifar10

D:\Users\ryanl\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [22]:
(x_train, x_test), (y_train, y_test) = cifar10.load_data()

In [45]:
def cifar_generator(image_array, batch_size=32):
    while True:
        for indexs in range(0, len(image_array), batch_size):
            images = x_train[indexs: indexs+batch_size]
            labels = y_train[indexs: indexs+batch_size]
            yield images, labels

In [ ]:
cifar_gen = cifar_generator(x_train)

In [ ]:
images, labels = next(cifar_gen)

In [ ]:
img_combine(images)

In [ ]:
images, labels = next(cifar_gen)

In [ ]:
img_combine(images)

## 可以看到兩次的圖片並不一樣，這樣就可以開始訓練囉！

## 作業

請參考昨天的程式碼，將訓練資料讀取方式改寫成 Generator，並將原本的 model.fit 改為 model.fit_generator 來進行訓練。請參考 Keras [官方文件中 fit_generator 的說明](https://keras.io/models/sequential/)

In [39]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop

from math import ceil

In [33]:
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練的 epochs 數量

# 讀取資料並檢視
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [38]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_8 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 15, 15, 64)        18496     
__________

In [44]:
cifar_gen = cifar_generator(image_array=x_train, batch_size=batch_size)

images, labels = next(cifar_gen)
print('images:', images.shape)
print('labels:', labels.shape)
print(labels[0][0][0])

images: (128, 32, 32, 3)
labels: (128, 32, 32, 3)
[0.61960787 0.4392157  0.19215687]


In [46]:
history = model.fit_generator(cifar_generator(image_array=x_train, batch_size=batch_size), 
                              validation_data=(x_test, y_test), steps_per_epoch=ceil(len(x_train) / batch_size), epochs=epochs)

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
391/391 [==============================] - 233s 595ms/step - loss: 1.7429 - acc: 0.3689 - val_loss: 1.4311 - val_acc: 0.4722
Epoch 2/10
391/391 [==============================] - 227s 580ms/step - loss: 1.3117 - acc: 0.5374 - val_loss: 1.1340 - val_acc: 0.5956
Epoch 3/10
391/391 [==============================] - 227s 580ms/step - loss: 1.1133 - acc: 0.6097 - val_loss: 0.9917 - val_acc: 0.6538
Epoch 4/10
391/391 [==============================] - 237s 605ms/step - loss: 0.9781 - acc: 0.6602 - val_loss: 0.9411 - val_acc: 0.6740
Epoch 5/10
391/391 [==============================] - 226s 578ms/step - loss: 0.8852 - acc: 0.6912 - val_loss: 0.7855 - val_acc: 0.7270
Epoch 6/10
391/391 [==============================] - 230s 587ms/step - loss: 0.8163 - acc: 0.7152 - val_loss: 0.7969 - val_acc: 0.7273
Epoch 7/10
391/391 [==============================] - 221s 565ms/step - loss: 0.7568 - acc: 0.7363 - val_loss: 0.7249 - val_acc: 0.7516
Epoch 8/10
391/391 [============================